In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'/home/arch_aditya/Desktop/text-summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir : Path
    STATUS_FILE : str
    ALL_REQUIRED_FILES : list

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager :
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_validation(self) -> DataValidationConfig:
          config = self.config.data_validation

          create_directories([config.root_dir])

          data_validation_config = DataValidationConfig(
                root_dir=config.root_dir,
                STATUS_FILE= config.STATUS_FILE,
                ALL_REQUIRED_FILES=config.ALL_REQUIRED_FILES
          )
          return data_validation_config

In [9]:
import os 
from textSummarizer.logging import logger

class DataValidation :
    def __init__(self, config : DataValidationConfig) :
        self.config = config
    
    def validate_all_files_exists(self) -> bool :
        try:
            validation_status = None
            all_files = os.listdir(os.path.join("artifacts","data_ingestion","samsum_dataset"))

            for file in all_files:
                if file not in self.config.ALL_REQUIRED_FILES :
                    validation_status = False
                    logger.error(f"File {file} not found in required files")
                    with open(self.config.STATUS_FILE, "w") as f :
                        f.write(f"validation_status = {validation_status}")
                else :
                    validation_status = True
                    logger.info(f"File {file} found in required files")
                    with open(self.config.STATUS_FILE, "w") as f :
                        f.write(f"validation_status = {validation_status}")
            return validation_status
        
        except Exception as e:
            logger.error(f"Error while validating all files exists : {e}")
            raise e

In [10]:
try :
    config = ConfigurationManager()
    data_validation = DataValidation(config=config.get_data_validation())
    data_validation.validate_all_files_exists()

except Exception as e:
    raise e

[2024-03-17 02:18:25,708 - INFO - common - 28 - yaml file : config/config.yaml loaded successfully]
[2024-03-17 02:18:25,709 - INFO - common - 28 - yaml file : params.yaml loaded successfully]
[2024-03-17 02:18:25,710 - INFO - common - 57 - Created directory artifacts]
[2024-03-17 02:18:25,710 - INFO - common - 57 - Created directory artifacts/data_validation]
[2024-03-17 02:18:25,711 - ERROR - 2245237591 - 16 - File dataset_dict.json not found in required files]
[2024-03-17 02:18:25,711 - INFO - 2245237591 - 21 - File test found in required files]
[2024-03-17 02:18:25,712 - INFO - 2245237591 - 21 - File train found in required files]
[2024-03-17 02:18:25,712 - INFO - 2245237591 - 21 - File validation found in required files]
